# Merging into Final Friction Surface(s)
</br> This notebook merges the on road and off road speed surfaces and recalculates them as friction surfaces

In [4]:
import os, sys

from datetime import datetime

import pprint
# from pprint import pprint

import common_rasterio_ops as rast_ops

import numpy as np
from numpy import pi, log, tan, empty, float32, arctan, rad2deg, gradient
from numpy import arctan2, reshape, where
from scipy.ndimage import gaussian_gradient_magnitude

import rasterio
from rasterio import features, transform
from rasterio.mask import mask
from rasterio.transform import Affine
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile

import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import shape, box, Polygon

### Setup

Directories

In [5]:
data_dir = r'P:\PAK\\Code\Accessibility\Inputs'
out_dir = r'P:\PAK\\Code\Accessibility\Intermediate_Rasters'
fric_dir = r'P:\PAK\\Code\Accessibility\Friction_Surface'

Projections

In [6]:
dest_crs = 'EPSG:32642' # this is a Pakistani UTM projection, assign correct projection for project area

Today's date

In [7]:
today = datetime.today().strftime('%y%m%d')

Data creation date

In [8]:
# Note this only works if all your data was produced on the same day. Otherwise you'll need to manually specify file dates in their names
data_date = '210927'

Resolution

In [9]:
# change to reflect the resolution of raster file you want to create
res = '31m'

# Create final multi-modal friction surface

Now that the walking surface is created, we can combine it with the previously created on-road speed surfaces, once we transform them to friction surfaces. Combining the two yields the final multi-modal friction surface

### Loading data

Load in a shapefile of KP to clip data with

In [10]:
aoi = gpd.read_file(r'P:\PAK\GEO\Boundaries\OCHA\pak_admbnda_adm1_ocha_pco_gaul_20181218.shp')

In [11]:
aoi = aoi[aoi['ADM1_EN'] == 'Khyber Pakhtunkhwa']
aoi = aoi.to_crs(dest_crs)

In [12]:
# Buffer the polygon by 20km so we take in nearby markets and roads that may be used
aoi.geometry = aoi.buffer(20000)

In [13]:
aoi.head()

,Shape_Leng,Shape_Area,ADM1_EN,ADM1_PCODE,ADM1_REF,ADM1ALT1EN,ADM1ALT2EN,ADM0_EN,ADM0_PCODE,date,validOn,validTo,geometry
2,27.479368,9.901186,Khyber Pakhtunkhwa,PK2,None,None,None,Pakistan,PK,2018-11-30,2018-12-18,None,"POLYGON ((505910.537 3561734.747, 505840.700 3..."


Load in the off-road friction surfaces (for all seasons).

In [14]:
with rasterio.open(os.path.join(fric_dir,f'KP_friction_walk_dry_{data_date}_{res}_masked.tif')) as dry_offr_src:
    friction_walk_dry_final = dry_offr_src.read()
    dry_offr_profile = dry_offr_src.profile
    dry_offr_transform = dry_offr_src.transform

with rasterio.open(os.path.join(fric_dir,f'KP_friction_walk_msn_{data_date}_{res}_masked.tif')) as msn_offr_src:
    friction_walk_msn_final = msn_offr_src.read()
    msn_offr_profile = msn_offr_src.profile
    msn_offr_transform = msn_offr_src.transform
    
with rasterio.open(os.path.join(fric_dir,f'KP_friction_walk_winter_{data_date}_{res}_masked.tif')) as winter_offr_src:
    friction_walk_winter_final = winter_offr_src.read()
    winter_offr_profile = winter_offr_src.profile
    winter_offr_transform = winter_offr_src.transform

Load in the vertical distance raster, clip it, and warp it to the correct resolution / cell size.

In [19]:
# This layer represents the extra total distance that must be traversed due to vertical gains, accounting for the likelihood of walkers using switchbacks on slopes above 30 degrees
# Swap in the non-switchback ("simple") layer produced in Step 2 if not using switchbacks
with rasterio.open(os.path.join(out_dir,f'KP_Combined_VertHorizontal_Distance_Switchbacks_220109_{res}_masked.tif'),'r') as vertdist_switchback_src:
    vert_dist_switchback = vertdist_switchback_src.read()

Load in the on-road speed surfaces (for all seasons).

In [20]:
with rasterio.open(os.path.join(out_dir,f'KP_OnRoad_dry_speed_211019_{res}_masked.tif')) as dry_onr_src:
    drivespeed_dry_array = dry_onr_src.read()
    dry_onr_profile = dry_onr_src.profile
    dry_onr_transform = dry_onr_src.transform

with rasterio.open(os.path.join(out_dir,f'KP_OnRoad_msn_speed_211019_{res}_masked.tif')) as msn_onr_src:
    drivespeed_msn_array = msn_onr_src.read()
    msn_onr_profile = msn_onr_src.profile
    msn_onr_transform = msn_onr_src.transform
    
with rasterio.open(os.path.join(out_dir,f'KP_OnRoad_winter_speed_211019_{res}_masked.tif')) as winter_onr_src:
    drivespeed_winter_array = winter_onr_src.read()
    winter_onr_profile = winter_onr_src.profile
    winter_onr_transform = winter_onr_src.profile

### Convert to friction surfaces

In [21]:
# use the offroad transform as the reference transform (this is arbitrary, any would do)
base_transform = dry_offr_transform

Convert speed surfaces to friction surfaces

In [22]:
# refactor driving speeds to friction values in units of cell size / hour (e.g. 30m / hour)
# to get values in minutes multiply by 60
friction_drive_dry_step1 = (1 / drivespeed_dry_array) / (1000 / base_transform.a)
friction_drive_msn_step1 = (1 / drivespeed_msn_array) / (1000 / base_transform.a)
friction_drive_winter_step1 = (1 / drivespeed_winter_array) / (1000 / base_transform.a)

C:\Users\wb467985\AppData\Local\Temp/ipykernel_13196/2999462858.py:3: RuntimeWarning: divide by zero encountered in true_divide
  friction_drive_dry_step1 = (1 / drivespeed_dry_array) / (1000 / base_transform.a)
C:\Users\wb467985\AppData\Local\Temp/ipykernel_13196/2999462858.py:4: RuntimeWarning: divide by zero encountered in true_divide
  friction_drive_msn_step1 = (1 / drivespeed_msn_array) / (1000 / base_transform.a)
C:\Users\wb467985\AppData\Local\Temp/ipykernel_13196/2999462858.py:5: RuntimeWarning: divide by zero encountered in true_divide
  friction_drive_winter_step1 = (1 / drivespeed_winter_array) / (1000 / base_transform.a)


Multiply these friction surfaces by the vertical+horizontal distance

In [24]:
# we use the simple vertical distance, without switchbacks, on the assumption road switchbacks are incorporated into the vector line objects that have been rasterized

friction_drive_dry_step2 = np.multiply(friction_drive_dry_step1,vert_dist_switchback)
friction_drive_msn_step2 = np.multiply(friction_drive_msn_step1,vert_dist_switchback)
friction_drive_winter_step2 = np.multiply(friction_drive_winter_step1,vert_dist_switchback)

Compute a final friction surface by taking the lowest (speediest) value from each raster. In almost all cases this will be the driving surface speed. Note that because our NoData value is `-99999` it will always be lowest, so we need to exclude this

In [ ]:
friction_dry_final = np.where(((friction_drive_dry_step2 < friction_walk_dry_final) | (friction_walk_dry_final == -99999)), friction_drive_dry_step2, friction_walk_dry_final)
friction_msn_final = np.where(((friction_drive_msn_step2 < friction_walk_msn_final) | (friction_walk_msn_final == -99999)), friction_drive_msn_step2, friction_walk_msn_final)
friction_winter_final = np.where(((friction_drive_winter_step2 < friction_walk_winter_final) | (friction_walk_winter_final == -99999)), friction_drive_winter_step2, friction_walk_winter_final)

In [ ]:
np.min(friction_winter_final)

In [ ]:
np.max(friction_winter_final)

## Export

### Adjustments to reduce final file size

Round up to 8 decimal points to reduce file size. Note this slightly reduces accuracy

In [ ]:
friction_dry_final = np.round(friction_dry_final,8).astype(np.float32)
friction_msn_final = np.round(friction_msn_final,8).astype(np.float32)
friction_winter_final = np.round(friction_winter_final,8).astype(np.float32)

In [ ]:
friction_winter_final.shape

### Remove index band

In [ ]:
friction_dry_final = friction_dry_final[0, :, :]
friction_msn_final = friction_msn_final[0, :, :]
friction_winter_final = friction_winter_final[0, :, :]

### Export

Modify the output profile to compress efficiently

In [37]:
export_profile = dry_offr_profile.copy()
export_profile.update({"dtype":'float32',\
                       "COMPRESS":'ZSTD',
                       "NUM_THREADS":'ALL_CPUS',
                       "nodata":-99999})

In [39]:
with rasterio.open(os.path.join(fric_dir,f'KP_friction_dry_{today}_{res}.tif'),'w',**export_profile) as dst:
    dst.write(friction_dry_final,indexes=1)
    dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software

with rasterio.open(os.path.join(fric_dir,f'KP_friction_msn_{today}_{res}.tif'),'w',**export_profile) as dst:
    dst.write(friction_msn_final,indexes=1)
    dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software

with rasterio.open(os.path.join(fric_dir,f'KP_friction_winter_{today}_{res}.tif'),'w',**export_profile) as dst:
    dst.write(friction_winter_final,indexes=1)
    dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software

### Optional -- Export cropped version

We crop the raster by our analysis area to reduce file sizes and computation time

In [ ]:
# Clip the friction arrays by the buffered KP outline
# use a MemoryFile to avoid lots of IO -- otherwise have to save down, then load back up.

# dry
friction_dry_final_mask, friction_dry_mask_tform = rast_ops.clip_in_memory(friction_dry_final,export_profile,aoi.geometry)

# monsoon
friction_msn_final_mask, friction_msn_mask_tform = rast_ops.clip_in_memory(friction_msn_final,export_profile,aoi.geometry)

# winter
friction_winter_final_mask, friction_winter_mask_tform = rast_ops.clip_in_memory(friction_winter_final,export_profile,aoi.geometry)


In [41]:
# ensure it's float32 so the output size is smaller
friction_dry_final_mask = friction_dry_final_mask.astype(np.float32)
friction_msn_final_mask = friction_msn_final_mask.astype(np.float32)
friction_winter_final_mask = friction_winter_final_mask.astype(np.float32)

In [42]:
# make a new profile for export, inserting the new height/width/transform values from the clipped raster

mask_profile = export_profile.copy()

mask_profile.update({"dtype" : 'float32',
                     "height":friction_msn_final_mask.shape[0],
                     "width":friction_msn_final_mask.shape[1],
                     "transform" : friction_msn_mask_tform})

In [44]:
# Export the three seasons

with rasterio.open(os.path.join(fric_dir,f'KP_friction_dry_{today}_{res}_masked.tif'),'w',**mask_profile) as dst:
    dst.write(friction_dry_final_mask,indexes=1)
    dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software
    
with rasterio.open(os.path.join(fric_dir,f'KP_friction_msn_{today}_{res}_masked.tif'),'w',**mask_profile) as dst:
    dst.write(friction_msn_final_mask,indexes=1)
    dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software

with rasterio.open(os.path.join(fric_dir,f'KP_friction_winter_{today}_{res}_masked.tif'),'w',**mask_profile) as dst:
    dst.write(friction_winter_final_mask,indexes=1)
    dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software

## Optional -- Export COG versions

COGs work better on the web, distribute better in Dask, and are smaller/slimmer in file size for large rasters. What's not to like? 

In [69]:
final_fric_dir = r'P:\PAK\Code\kpgit_robert\data\friction'

In [70]:
fric_tifs = sorted([os.path.join(final_fric_dir,f'{res}',file) \
            for file \
            in os.listdir(os.path.join(final_fric_dir,f'{res}')) \
            if file.endswith(f"{res}_masked.tif")])


In [71]:
fric_tifs

['P:\\PAK\\Code\\kpgit_robert\\data\\friction\\31m\\KP_friction_dry_211019_31m_masked.tif',
 'P:\\PAK\\Code\\kpgit_robert\\data\\friction\\31m\\KP_friction_msn_211019_31m_masked.tif',
 'P:\\PAK\\Code\\kpgit_robert\\data\\friction\\31m\\KP_friction_winter_211019_31m_masked.tif']

In [72]:
for fric_lyr in fric_tifs:
    fric_lyr_cog = str.replace(fric_lyr,'.tif','_COG.tif')
    
    print(fric_lyr)
    !rio cogeo create {fric_lyr} {fric_lyr_cog}

P:\PAK\Code\kpgit_robert\data\friction\31m\KP_friction_dry_211019_31m_masked.tif
P:\PAK\Code\kpgit_robert\data\friction\31m\KP_friction_msn_211019_31m_masked.tif


Reading input: P:/PAK/Code/kpgit_robert/data/friction/31m/KP_friction_dry_211019_31m_masked.tif
Adding overviews...
Updating dataset tags...
Writing output to: \\sarpov\sarpov\PAK\Code\kpgit_robert\data\friction\31m\KP_friction_dry_211019_31m_masked_COG.tif


P:\PAK\Code\kpgit_robert\data\friction\31m\KP_friction_winter_211019_31m_masked.tif


Reading input: P:/PAK/Code/kpgit_robert/data/friction/31m/KP_friction_msn_211019_31m_masked.tif
Adding overviews...
Updating dataset tags...
Writing output to: \\sarpov\sarpov\PAK\Code\kpgit_robert\data\friction\31m\KP_friction_msn_211019_31m_masked_COG.tif
Reading input: P:/PAK/Code/kpgit_robert/data/friction/31m/KP_friction_winter_211019_31m_masked.tif
Adding overviews...
Updating dataset tags...
Writing output to: \\sarpov\sarpov\PAK\Code\kpgit_robert\data\friction\31m\KP_friction_winter_211019_31m_masked_COG.tif
